In [1]:
solver_grammar = """
    start: _NL? problem
    problem: ([expr | decl] _NL)+
    decl: symbol ":=" expr
    expr: binoperation
        | method
        | symbol
        | literal
    method: symbol "(" parameter ")"
    parameter: expr ("," expr)*
    binoperation: expr OPERATOR expr
    symbol: SYMBOL
    literal: LITERAL
    
    SYMBOL: /[a-zA-Z][a-zA-Z0-9]*/
    LITERAL: /\d*\.?\d+/
    OPERATOR: /([\/*\-+]|==)/
    
    %import common.INT -> NUMBER
    %import common.NEWLINE -> _NL
    %import common.WS_INLINE
    
    %ignore WS_INLINE
"""

In [13]:
from lark import Lark
parser = Lark(solver_grammar)
sample1 = """
t := triangle(a, b, c)
perimeter(t, 2)
solve(2 * perimeter(t))
"""
sample = """
t := triangle(a, b, c)
d := triangle(z, x, y)
perimeter(d, 3)
perimeter(t, 2 * perimeter(d))
dist(a, b) == 2
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample2 = """
t := triangle(a, b, c)
perimeter(t, 2 * 2)
eq(dist(a, b), 1)
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample3 = """
t := triangle(a,b,c)
perimeter(t, 5)
dist(a,c) == dist(b,c)
eq(dist(a,b),1)
solve(dist(a,c))
"""


print(parser.parse(sample1).pretty())
print(parser.parse(sample1))
str(parser.parse(sample1).children[0].children[0].children[0].children[0])

start
  problem
    decl
      symbol	t
      expr
        method
          symbol	triangle
          parameter
            expr
              symbol	a
            expr
              symbol	b
            expr
              symbol	c
    expr
      method
        symbol	perimeter
        parameter
          expr
            symbol	t
          expr
            literal	2
    expr
      method
        symbol	solve
        parameter
          expr
            binoperation
              expr
                literal	2
              *
              expr
                method
                  symbol	perimeter
                  parameter
                    expr
                      symbol	t

Tree(start, [Tree(problem, [Tree(decl, [Tree(symbol, [Token(SYMBOL, 't')]), Tree(expr, [Tree(method, [Tree(symbol, [Token(SYMBOL, 'triangle')]), Tree(parameter, [Tree(expr, [Tree(symbol, [Token(SYMBOL, 'a')])]), Tree(expr, [Tree(symbol, [Token(SYMBOL, 'b')])]), Tree(expr, [Tree(symbol, [Token(SYMBOL, 'c')

't'

In [14]:
from problem import *
from lark import Tree, Token

p = Problem() 

methods = {
    'triangle': p.triangle,
    'perimeter': p.perimeter,
    'eq': p.eq,
    'dist': p.dist,
    'solve': p.solve,
    '+': p.add,
    '-': p.sub,
    '*': p.mul,
    '/': p.div,
    '==': p.eq
}

def process_expression(root, problem):
    if root.data == 'literal':
        return(float(root.children[0]))
    elif root.data == 'symbol':
        return(str(root.children[0]))
    elif root.data == 'binoperation':
        res = visit_branch(root, problem) 
        return (methods.get(res[1], lambda: 'Invalid')(res[0], res[2]))
    elif root.data == 'method':
        method = visit_branch(root, problem)
        return(methods.get(method[0], lambda: 'Invalid')(*method[1]))

    #return visit_branch(root)

def visit_branch(root, problem):
    if isinstance(root, Tree):
        params = [visit_branch(child, problem) for child in root.children]
       # print(params)
        if root.data == 'expr':
            return process_expression(root.children[0], problem)
        elif root.data == 'decl':
            problem.bind(*params)
        elif root.data == 'literal' or root.data == 'symbol':
            return params[0]
        return params
    if isinstance(root, Token):
        return str(root)
               
visit_branch(parser.parse(sample1), p)

[[['t', <problem.Triangle at 0x1ba1db1f0b8>], None, None]]